In [20]:
import json
from matplotlib import pyplot as plt
import numpy as np

In [4]:
hosts = [ "niagara", "nile", "kolyma", "euphrates", ]
site = [ "local", "nfs" ]
bs = [ "4k", "16k", "64k", "256k", "1m", ]
jobs = [ 1, 2, 4, 8, 16, ]
file_size = [ "1G", "5G"]

In [ ]:
def create_graph(host, site, bs, jobs, fsize):
    modes = [ "rand_write", "rand_read", "seq_write", "seq_read", "mixed_rw" ]
    plot_data = {}
    for m in modes:
        with open(f"results/{host}_{site}_bs{bs}_jobs{jobs}_size{fsize}_{m}.txt") as fd:
            data = json.load(fd)
            if m.endswith("write"):
                plot_data[m] = float(data["jobs"][0]["write"]["bw"])/1024
            elif m.endswith("read"):
                plot_data[m] = float(data["jobs"][0]["read"]["bw"])/1024
            else:
                plot_data["mixed_read"] = float(data["jobs"][0]["read"]["bw"])/1024
                plot_data["mixed_write"] = float(data["jobs"][0]["write"]["bw"])/1024

    print(plot_data)

    width = 0.25
    x = np.arange(3)*0.6
    seqrand = ["Sequential", "Random", "Mixed"]

    name = {
        "Read": ["seq_read", "rand_read", "mixed_read"],
        "Write": ["seq_write", "rand_write", "mixed_write"],
    }

    fig, ax = plt.subplots(layout="constrained", figsize=(10, 5))
    for i, (key, value) in enumerate(name.items()):
        rects = ax.bar(x + i*width, [plot_data[v] for v in value], width=width, label=key)
        ax.bar_label(rects, padding=3)

    ax.set_ylabel('Bandwidth (MB/s)')
    ax.set_title(f"{host} {site} bs:{bs} jobs:{jobs} size:{fsize}")
    ax.set_xticks(x + width/2, seqrand)
    ax.legend(loc='best', ncols=3)
    ax.set_ylim(0)

    fig.show()

host = "niagara"
site = "local"
bs = "4k"
jobs = 1
fsize = "1G"

create_graph("niagara", site, bs, jobs, fsize)
create_graph("euphrates", site, bs, jobs, fsize)
create_graph("kolyma", site, bs, jobs, fsize)
create_graph("nile", site, bs, jobs, fsize)